<a href="https://colab.research.google.com/github/bahadirbesirkestane/Staj/blob/main/ElasticSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Elasticsearch**
Eğer büyük veri blokları arasında metin arama (full text search) gibi bir ihtiyacımız varsa, bu durumda Elasticsearch bizim için doğru bir tercih olabilir.

Elasticsearch ; text üzerinden doğrudan arama yapmak yerine, indexler üzerinden arama yapar ve çok hızlı bir şekilde sonuçlar üretir. Bunun yanında sorgular üzerinde istatiksel analizler , skorlamalar da yapabilir


In [16]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

>tensorflow-io ve elasticsearch paketlerinin indirilmesi.

In [1]:
!pip install tensorflow-io
!pip install elasticsearch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.8/28.8 MB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 70.1 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-io-gcs-filesystem
    Found existing installation: tensorflow-io-gcs-filesystem 0.33.0
    Uninstalling tensorflow-io-gcs-filesystem-0.33.0:
      Successfully uninstalled tensorflow-io-gcs-filesystem-0.33.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.5/395.5 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 15.8 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.4
    Uninstalling urllib3-2.0.4:
      Successfully uninstalled urllib3-2.0.4


>Paketlerin yüklenmesi.

In [2]:
import os
import time
from sklearn.model_selection import train_test_split
from elasticsearch import Elasticsearch
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
import tensorflow_io as tfio

>İmport edilen paketin doğrulanması.

In [3]:
print("tensorflow-io version: {}".format(tfio.__version__))
print("tensorflow version: {}".format(tf.__version__))

tensorflow-io version: 0.34.0
tensorflow version: 2.13.0


###**Elasticsearcın bir demo versiyonunun kuruluması**

In [4]:
%%bash

wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
wget -q https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512
tar -xzf elasticsearch-oss-7.9.2-linux-x86_64.tar.gz
sudo chown -R daemon:daemon elasticsearch-7.9.2/
shasum -a 512 -c elasticsearch-oss-7.9.2-linux-x86_64.tar.gz.sha512

elasticsearch-oss-7.9.2-linux-x86_64.tar.gz: OK


>Gelen örneğin deamon işlemş olarak çalıştırılması

In [5]:
%%bash --bg

sudo -H -u daemon elasticsearch-7.9.2/bin/elasticsearch

In [6]:
# Örneğin başlamasına izin vermek için birkaç saniye beklenmelidir.
time.sleep(20)

>Örnek başlatıldığında kullanılabilirliği doğrulamak için elasticsearch grep yapılır.
>
>Not : Grep, verilen bir yazidan belirli kriterler dahilinde parcalar cikarir.

In [7]:
%%bash

ps -ef | grep elasticsearch

root         481     479  0 08:45 ?        00:00:00 sudo -H -u daemon elasticsearch-7.9.2/bin/elasticsearch
daemon       482     481 84 08:45 ?        00:00:16 /content/elasticsearch-7.9.2/jdk/bin/java -Xshare:auto -Des.networkaddress.cache.ttl=60 -Des.networkaddress.cache.negative.ttl=10 -XX:+AlwaysPreTouch -Xss1m -Djava.awt.headless=true -Dfile.encoding=UTF-8 -Djna.nosys=true -XX:-OmitStackTraceInFastThrow -XX:+ShowCodeDetailsInExceptionMessages -Dio.netty.noUnsafe=true -Dio.netty.noKeySetOptimization=true -Dio.netty.recycler.maxCapacityPerThread=0 -Dio.netty.allocator.numDirectArenas=0 -Dlog4j.shutdownHookEnabled=false -Dlog4j2.disable.jmx=true -Djava.locale.providers=SPI,COMPAT -Xms1g -Xmx1g -XX:+UseG1GC -XX:G1ReservePercent=25 -XX:InitiatingHeapOccupancyPercent=30 -Djava.io.tmpdir=/tmp/elasticsearch-13713732631798129154 -XX:+HeapDumpOnOutOfMemoryError -XX:HeapDumpPath=data -XX:ErrorFile=logs/hs_err_pid%p.log -Xlog:gc*,gc+age=trace,safepoint:file=logs/gc.log:utctime,pid,tags:fileco

>Küme bilgisini alabilmek için temel sorgulama.

In [8]:
%%bash

curl -sX GET "localhost:9200/"

{
  "name" : "066fd788e202",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "EspwxuSoTLib7suDrdIXQw",
  "version" : {
    "number" : "7.9.2",
    "build_flavor" : "oss",
    "build_type" : "tar",
    "build_hash" : "d34da0ea4a966c4e49417f2da2f244e3e97b4e6e",
    "build_date" : "2020-09-23T00:45:33.626720Z",
    "build_snapshot" : false,
    "lucene_version" : "8.6.2",
    "minimum_wire_compatibility_version" : "6.8.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


###**Veriseti**
PetFinder verisetinden elasticsearch ile zenginleştirip, sonuç tahmini olarak hayvan sahiplenip sahiplenmeyeceği bilgisini almak istiyoruz.

In [9]:
dataset_url = 'http://storage.googleapis.com/download.tensorflow.org/data/petfinder-mini.zip'
csv_file = 'datasets/petfinder-mini/petfinder-mini.csv'
tf.keras.utils.get_file('petfinder_mini.zip', dataset_url,
                        extract=True, cache_dir='.')
pf_df = pd.read_csv(csv_file)

1668792/1668792 [==============================] - 0s 0us/step


In [10]:
pf_df.head()

,Type,Age,Breed1,Gender,Color1,Color2,MaturitySize,FurLength,Vaccinated,Sterilized,Health,Fee,Description,PhotoAmt,AdoptionSpeed
0,Cat,3,Tabby,Male,Black,White,Small,Short,No,No,Healthy,100,Nibble is a 3+ month old ball of cuteness. He ...,1,2
1,Cat,1,Domestic Medium Hair,Male,Black,Brown,Medium,Medium,Not Sure,Not Sure,Healthy,0,I just found it alone yesterday near my apartm...,2,0
2,Dog,1,Mixed Breed,Male,Brown,White,Medium,Medium,Yes,No,Healthy,0,Their pregnant mother was dumped by her irresp...,7,3
3,Dog,4,Mixed Breed,Female,Black,Brown,Medium,Short,Yes,No,Healthy,150,"Good guard dog, very alert, active, obedience ...",8,2
4,Dog,1,Mixed Breed,Male,Black,No Color,Medium,Short,No,No,Healthy,0,This handsome yet cute boy is up for adoption....,3,2


>0 hayvanın sahiplenilmediğini, 1 ise hayvanın sahiplenildiğini gösterir.

In [11]:
# Orijinal veri setinde "4" evcil hayvanın sahiplenilmediğini gösterir.
pf_df['target'] = np.where(pf_df['AdoptionSpeed']==4, 0, 1)

# Kullanılmayan kolonların silinmesi.
pf_df = pf_df.drop(columns=['AdoptionSpeed', 'Description'])

In [12]:
# Veri noktası ve sütun sayısı
len(pf_df), len(pf_df.columns)

(11537, 14)

>Verisetinin eğitim ve test için bölünmesi.

In [13]:
train_df, test_df = train_test_split(pf_df, test_size=0.3, shuffle=True)
print("Number of training samples: ",len(train_df))
print("Number of testing sample: ",len(test_df))

Number of training samples:  8075
Number of testing sample:  3462


>Eğitim ve test verileri elasticsearch indexlerinde saklanır.
>
>Verilerin yerel elasticsearch kümesinde saklanması, eğitim ve çıkarım amacıyla sürekli olarak uzaktan veri alımına yönelik bir ortamı simüle eder.

In [14]:
ES_NODES = "http://localhost:9200"

def prepare_es_data(index, doc_type, df):
  records = df.to_dict(orient="records")
  es_data = []
  for idx, record in enumerate(records):
    meta_dict = {
          "index": {
              "_index": index,
              "_type": doc_type,
              "_id": idx
          }
      }
    es_data.append(meta_dict)
    es_data.append(record)

  return es_data

def index_es_data(index, es_data):
  es_client = Elasticsearch(hosts = [ES_NODES])
  if es_client.indices.exists(index):
      print("deleting the '{}' index.".format(index))
      res = es_client.indices.delete(index=index)
      print("Response from server: {}".format(res))

  print("creating the '{}' index.".format(index))
  res = es_client.indices.create(index=index)
  print("Response from server: {}".format(res))

  print("bulk index the data")
  res = es_client.bulk(index=index, body=es_data, refresh = True)
  print("Errors: {}, Num of records indexed: {}".format(res["errors"], len(res["items"])))

In [ ]:
train_es_data = prepare_es_data(index="train", doc_type="pet", df=train_df)
test_es_data = prepare_es_data(index="test", doc_type="pet", df=test_df)

index_es_data(index="train", es_data=train_es_data)
time.sleep(3)
index_es_data(index="test", es_data=test_es_data)

>Tfio verisetinin hazırlanması.
>
>Veriler kümede mevcut olduğunda, verilerin endekslerden akışını sağlamak için yalnızca tensorflow-io gerekir. Elasticsearch.ElasticsearchIODataset sınıfından bu amaç için yararlanılır.

###**Verisetinin eğitilmesi**

In [ ]:
BATCH_SIZE=32
HEADERS = {"Content-Type": "application/json"}

train_ds = tfio.experimental.elasticsearch.ElasticsearchIODataset(
        nodes=[ES_NODES],
        index="train",
        doc_type="pet",
        headers=HEADERS
    )

# Prepare a tuple of (features, label)
train_ds = train_ds.map(lambda v: (v, v.pop("target")))
train_ds = train_ds.batch(BATCH_SIZE)

>Test

In [ ]:
test_ds = tfio.experimental.elasticsearch.ElasticsearchIODataset(
        nodes=[ES_NODES],
        index="test",
        doc_type="pet",
        headers=HEADERS
    )

# Prepare a tuple of (features, label)
test_ds = test_ds.map(lambda v: (v, v.pop("target")))
test_ds = test_ds.batch(BATCH_SIZE)

###**Keras önişleme katmanları**
Yapılandırılmış veri eğitimine göre, daha sezgisel oldukları ve modellerle kolayca entegre edilebildikleri için Keras Ön İşleme Katmanlarının kullanılması tavsiye edilir.

In [19]:
def get_normalization_layer(name, dataset):
  # Normalleştirme katmanı oluşturulması.
  normalizer = preprocessing.Normalization()

  # Yalnızca özelliğimizi veren bir Veri Seti hazırlanması.
  feature_ds = dataset.map(lambda x, y: x[name])

  # Verilerin istatistikleri
  normalizer.adapt(feature_ds)

  return normalizer

def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
  # Dizeleri tamsayı indekslerine dönüştürecek bir StringLookup katmanının oluşturulması.
  if dtype == 'string':
    index = preprocessing.StringLookup(max_tokens=max_tokens)
  else:
    index = preprocessing.IntegerLookup(max_values=max_tokens)

  # Verilen parametreye göre Veri Seti hazırlanması.
  feature_ds = dataset.map(lambda x, y: x[name])

  # Olası değerler kümesi
  index.adapt(feature_ds)

  # Tamsayı indexlerinin belirlenmesi.
  encoder = preprocessing.CategoryEncoding(max_tokens=index.vocab_size())

  # İndexe göre Veri Seti hazırlanması.
  feature_ds = feature_ds.map(index)

  # Olası indexlerin uzayı.
  encoder.adapt(feature_ds)

  return lambda feature: encoder(index(feature))

>Örnek bir kaydın özelliklerinin gözlemlenmesi.

In [ ]:
ds_iter = iter(train_ds)
features, label = next(ds_iter)
{key: value.numpy()[0] for key,value in features.items()}

>Bir özellik altkümesinin seçilmesi.

In [ ]:
all_inputs = []
encoded_features = []

# Sayısal özellikler.
for header in ['PhotoAmt', 'Fee']:
  numeric_col = tf.keras.Input(shape=(1,), name=header)
  normalization_layer = get_normalization_layer(header, train_ds)
  encoded_numeric_col = normalization_layer(numeric_col)
  all_inputs.append(numeric_col)
  encoded_features.append(encoded_numeric_col)


categorical_cols = ['Type', 'Color1', 'Color2', 'Gender', 'MaturitySize',
                    'FurLength', 'Vaccinated', 'Sterilized', 'Health', 'Breed1']
for header in categorical_cols:
  categorical_col = tf.keras.Input(shape=(1,), name=header, dtype='string')
  encoding_layer = get_category_encoding_layer(header, train_ds, dtype='string',
                                               max_tokens=5)
  encoded_categorical_col = encoding_layer(categorical_col)
  all_inputs.append(categorical_col)
  encoded_features.append(encoded_categorical_col)

###**Modelin eğitilmesi**

In [22]:
# Parametrelerin ayarlanması.

OPTIMIZER="adam"
LOSS=tf.keras.losses.BinaryCrossentropy(from_logits=True)
METRICS=['accuracy']
EPOCHS=10

In [ ]:
# Kolonları tf.keras katmanına dönüştürülmesi.
all_features = tf.keras.layers.concatenate(encoded_features)

# Modelin inşa edilmesi.
x = tf.keras.layers.Dense(32, activation="relu")(all_features)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(64, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(1)(x)
model = tf.keras.Model(all_inputs, output)

tf.keras.utils.plot_model(model, rankdir='LR', show_shapes=True)

In [ ]:
# Modeli derleme
model.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=METRICS)

In [ ]:
# Modelin fit edilmesi.
model.fit(train_ds, epochs=EPOCHS)

###**Sonuç**

In [ ]:
res = model.evaluate(test_ds)
print("test loss, test acc:", res)